<h1>Exercise 49</h1>
<p>Write a query to find all couples of trade for same stock that happened in the range of 10 seconds and having the difference by more than 10%. Output result should also list the percentage of price difference between the 2 trade.</p>
<strong>Expected outcome:</strong>
<br>
<img src="Day-49-Expected_Output.png" width="450">


In [2]:
import os
import sqlite3
import pandas as pd

db_name="exercise49.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS Trade_tbl(TRADE_ID VARCHAR(20), Trade_Timestamp TIME, Trade_Stock VARCHAR(20),Quantity INT, Price FLOAT)""")
conn.commit()

trade_data=[
    ('TRADE1','10:01:05','ITJunction4All',100,20),
    ('TRADE2','10:01:06','ITJunction4All',20,15),
    ('TRADE3','10:01:08','ITJunction4All',150,30),
    ('TRADE4','10:01:09','ITJunction4All',300,32),
    ('TRADE5','10:10:00','ITJunction4All',-100,19),
    ('TRADE6','10:10:01','ITJunction4All',-300,19)
]
cursor.executemany("INSERT INTO Trade_tbl VALUES(?,?,?,?,?)",trade_data)
conn.commit()

querytoexec="SELECT * FROM Trade_tbl"
df=pd.read_sql(querytoexec,conn)
print(df)

  TRADE_ID Trade_Timestamp     Trade_Stock  Quantity  Price
0   TRADE1        10:01:05  ITJunction4All       100   20.0
1   TRADE2        10:01:06  ITJunction4All        20   15.0
2   TRADE3        10:01:08  ITJunction4All       150   30.0
3   TRADE4        10:01:09  ITJunction4All       300   32.0
4   TRADE5        10:10:00  ITJunction4All      -100   19.0
5   TRADE6        10:10:01  ITJunction4All      -300   19.0


In [3]:
querytoexec="SELECT t1.Trade_stock, t1.Trade_ID, t2.Trade_ID, t1.Trade_Timestamp, t2.Trade_Timestamp, t1.Price, t2.Price, ABS(t1.Price - t2.Price) * 1.0 / t1.Price * 100 FROM Trade_tbl t1 INNER JOIN Trade_tbl t2 ON t1.Trade_Stock = t2.Trade_Stock WHERE t1.Trade_Timestamp < t2.Trade_Timestamp AND (unixepoch(t2.Trade_Timestamp) - unixepoch(t1.Trade_Timestamp)) < 10 AND ABS(t1.Price - t2.Price) * 1.0 / t1.Price * 100 > 10 ORDER BY t1.Trade_ID"
df=pd.read_sql(querytoexec,conn)
print(df)
conn.close()

      Trade_Stock TRADE_ID TRADE_ID Trade_Timestamp Trade_Timestamp  Price  \
0  ITJunction4All   TRADE1   TRADE2        10:01:05        10:01:06   20.0   
1  ITJunction4All   TRADE1   TRADE3        10:01:05        10:01:08   20.0   
2  ITJunction4All   TRADE1   TRADE4        10:01:05        10:01:09   20.0   
3  ITJunction4All   TRADE2   TRADE3        10:01:06        10:01:08   15.0   
4  ITJunction4All   TRADE2   TRADE4        10:01:06        10:01:09   15.0   

   Price  ABS(t1.Price - t2.Price) * 1.0 / t1.Price * 100  
0   15.0                                        25.000000  
1   30.0                                        50.000000  
2   32.0                                        60.000000  
3   30.0                                       100.000000  
4   32.0                                       113.333333  
